In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in Audio Features

In [2]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

#### Drops all uncommon songs

In [3]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')

#### Ordering tracks

In [4]:
Compare = Compare.sort_values('track_uri',ascending=True)

In [5]:
Compare = Compare.reset_index().drop(['index'],axis=1)

In [6]:
len(Compare)

2254670

#### Creating Dataframe 'Cosine'

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
cosine = pd.read_pickle('../Pickle_Files/cosine_dataframe.pkl')
cosine = cosine.drop(['principal component 1'],axis=1)

In [9]:
X = cosine.values

#### Normalizing values before cosine calculated

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit(cosine.values).transform(cosine.values)

# Recommender System
<br>-  This Recommender System takes in an input dataframe and makes a recommendation using the cosine similarity of the provided features.
<br>-  It chooses N recommendations of the length of the training set and uses ".sample()" to pick a random number of recommendations the same length as the test set.
<br>-  It then calculates the precision and recall of the test set.

The definition below calculates the precision and recall of a given recommender system

In [11]:
def recommender_precision(predicted, actual):
    def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted)), 4)
        return prec

    precision = np.mean(list(map(calc_precision, predicted, actual)))
    return precision


def recommender_recall(predicted, actual):
    def calc_recall(predicted, actual):
        reca = [value for value in predicted if value in actual]
        reca = np.round(float(len(reca)) / float(len(actual)), 4)
        return reca

    recall = np.mean(list(map(calc_recall, predicted, actual)))
    return recall

#### Creating Two Empty Lists, Precision and Recall
<br> These are used in the Recommender System for calculations

In [12]:
Precision = []
Recall = []

In [13]:
def Recommender_System(df):
    
    # Obtaining index values to later compare 
    
    df = pd.merge(df,Compare['track_uri'].reset_index(),on='track_uri')
    
    #df = df.set_index('index')

    #df = df.reset_index()

    df = df[['track_uri','index']]

    df = pd.merge(cosine,df,on='track_uri')

    df = df.set_index('index')

    df = df.drop('track_uri',axis=1)

    df_values = df.index.values

    # Creates training and test set of .7 and .3

    msk = np.random.rand(len(df)) < 0.7

    dftrain = df[msk]

    dftest = df[~msk]

    dfvalues = dftrain.index.values # Index values used in cosine similarity
    
    num_tracks_desired = 1 ## Recommends only one song
    def rec_top_n(n):
        return np.argsort(cosine_similarity(X, X[dfvalues]), axis=0)[::-1][1:n] # Cosine similarity finds closest to each song
    # Prints index of all recommended songs
    print(rec_top_n(num_tracks_desired+1))
    test = rec_top_n(num_tracks_desired+1).flatten()

    test = Compare.iloc[test]

    testset = Compare[['track_name','track_uri','artist_name']].iloc[dftest.index]
    
    prediction = test.sample(len(dftest),replace=True)
    prediction = prediction[['track_name','track_uri','artist_name']]
    prediction.columns = ['track_name_pred', 'track_uri_pred','artist_name_pred']

    # Evalution

    pred = [prediction.index.values]
    # Uses index to check predicted against actual
    actual = [dftest.index.values] 

    #returns Precision 
    print('\nPrecision: ' +str(recommender_precision(pred,actual)))
    Precision.append(recommender_precision(pred,actual))
    #returns Recall
    print('\nRecall: ' +str(recommender_recall(pred,actual)))
    Recall.append(recommender_recall(pred,actual))
    
    # Returns songs predicted
    
    prediction = prediction.reset_index()
    testset = testset.reset_index()
    prediction = prediction.drop(['index'],axis=1)
    testset = testset.drop(['index'],axis=1)
    final = testset.join(prediction)
    return final

### Example Recommending for Playlist 0 (pid0)

In [23]:
pid0 = track_elems[track_elems['pid']==0]

In [24]:
pid0 = pd.merge(Compare,pid0[['pid','track_uri']],on='track_uri')

In [25]:
df = pid0

In [26]:
Recommender_System(pid0)

[[2159294  183262  642641 1002736  798025  582340  219080 1172906 1129520
   507604 1689282 2168460 1163516 1918340  347003 1288681 1469625  524616
  1191103  748905 1529666 2185974  741828 1423628 1764427  748905 1205303
  1265220  704634 1707156 1081190  471310  654369 2171462  488065 2028456
  1372072  295130 1393259 1986860]]

Precision: 0.0

Recall: 0.0


,track_name,track_uri,artist_name,track_name_pred,track_uri_pred,artist_name_pred
0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Hurricane - Arty Remix,spotify:track:65n4DkUhKUO4tECIabKwy1,Halsey
1,Where Is The Love?,spotify:track:0uqPG793dkDDN7sCUJJIVC,The Black Eyed Peas,Wherever I Go,spotify:track:46jLy47W8rkf8rEX04gMKB,OneRepublic
2,Ice Box,spotify:track:1Bv0Yl01xBDZD4OQP93fyl,Omarion,Rockin’,spotify:track:5HR4CYT5nPpFy8m9wJQgrr,The Weeknd
3,Year 3000,spotify:track:1TfAhjzRBWzYZ8IdUV3igl,Jonas Brothers,Saint Pablo,spotify:track:1o0kWPueYo94LIjPYOE5Nf,Kanye West
4,Livin on Sunday,spotify:track:1X5WGCrUMuwRFuYU1eAo2I,Cris Cab,I'd Rather Be With You,spotify:track:42rFOQqHmXf9IkdwVqkooK,Bootsy Collins
5,Leavin',spotify:track:20ORwCJusz4KS2PbTPVNKo,Jesse McCartney,In My Business,spotify:track:4Rab4O0lTJRtfnYK13jptm,Whitney Houston
6,Closer,spotify:track:2nbClS09zsIAqNkshg6jnp,Ne-Yo,The Funeral,spotify:track:4o0NjemqhmsYLIMwlcosvW,Band of Horses
7,How Do You Sleep? - Featuring Ludacris,spotify:track:3jkdQNkDTxxXtjSO4l0o1H,Jesse McCartney,Unsteady,spotify:track:7lGKEWMXVWWTt3X71Bv44I,X Ambassadors
8,This Is Me,spotify:track:5c1sfI6wIQEsSUw0xrkFdl,Demi Lovato,What Makes You Beautiful,spotify:track:3SPDQfj2UfWq6A2NllZnzn,One Direction
9,A Thousand Miles,spotify:track:5y69gQtK33qxb8a24ACkCy,Vanessa Carlton,Raining Men,spotify:track:1gCWLA71vXQmINwo6WxH4t,Rihanna


### Example with Popular DJs
<a id='pop_djs'></a>

In [14]:
one = Compare[Compare['artist_name']=='Avicii'].sample(5) 
two = Compare[Compare['artist_name']=='Hardwell'].sample(5)   
three = Compare[Compare['artist_uri']==('spotify:artist:2CIMQHirSU0MQqyYHq0eOx')].sample(5)#deadmau5
four = Compare[Compare['artist_name']=='Swedish House Mafia'].sample(5)   
five = Compare[Compare['artist_name']=='Calvin Harris'].sample(5)  

In [15]:
cols = five.columns

all_songs = pd.concat([one.head(5), two.head(5),three.head(5),four.head(5),five.head(5)], ignore_index=False)

In [16]:
all_songs.sample(5)

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name
367059,spotify:track:1GGyYXz9aP9yj4FX8neC81,I Need Your Love,spotify:album:2Sz9deA3jkua9Rkc5PdnEa,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,290986,None,I Need Your Love - Jacob Plant Remix
80341,spotify:track:0H7CznBypEWBuITBO1XOw4,I Created Disco,spotify:album:507DSnX3hTUOiBiBEDgwfx,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,228280,None,Neon Rocks
1679181,spotify:track:5nSeHqFqlNdSCdj2PjYG3b,I Created Disco,spotify:album:507DSnX3hTUOiBiBEDgwfx,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,258706,None,Love Souvenir
1825257,spotify:track:6Imb4OdvfMlMdn8dT8COsj,Beta,spotify:album:1kN0cNijvdJK8aFKBTZcIz,Hardwell,spotify:artist:6BrvowZBreEkXzJQMpL174,350600,None,Beta - Original Mix
223970,spotify:track:0lsqtZVyovtBSQtWdD44LH,18 Months,spotify:album:4TjPTVBmdyaowp08j2toa0,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,212560,None,Sweet Nothing


<br>We can see that these recommendations are very good
<br>The recommender as returned songs from Boregore (no.1) another popular DJ
<br>It has also recommended a song by one of the main DJs Calvin Harris showing these results are good

In [17]:
Recommender_System(all_songs)

[[1995809 1127335 1399949 1248946  846033  797561  856443 1647125 1239113
  1812137  909897  340309  120082 2004322]]

Precision: 0.0

Recall: 0.0


,track_name,track_uri,artist_name,track_name_pred,track_uri_pred,artist_name_pred
0,Don't You Worry Child,spotify:track:18oVklSAWDCvi1tNqpluY2,Swedish House Mafia,Be Ourselves,spotify:track:2kLBU2xvrwS2LVmoAYCgHm,Borgore
1,Move for Me - Mind Electric Mix,spotify:track:3Cf5X7eWD2nDs1NBJIWFyy,deadmau5,RGV,spotify:track:2wz2KOgnMg3EYhQ7Hnp2CJ,Carnage
2,Malo - Sgt. Slick Remix,spotify:track:3RouG6s20ZCWbgtFLO93G5,Avicii,One Type of Dark - Ta-ku Remix,spotify:track:6vMIGFY2ZV91q3RBkgTJUo,Ginger & The Ghost
3,Sweet Dreams - Avicii Swede Radio Edit,spotify:track:3geEYxtQEE29yRjXu654sD,Avicii,The Anthem,spotify:track:1Aa0edEUZ8sZdz33WVtjXI,Planetshakers
4,Creatures Of The Night - Sebastien Extended Mix,spotify:track:4pA9RsquXq7waJS3ElTVbO,Hardwell,Blessed Are the Ties That Bind,spotify:track:3t6SbUte9unLU7HFzRkK68,Graveyard Lovers
5,Antidote,spotify:track:4zMU6U7mZtx79bWkfG5DLe,Swedish House Mafia,A.U.M.,spotify:track:38TBD7Z9BNaASIcQNqcLXC,MC YOGI
6,One - Original Mix,spotify:track:5bcNxdOgueU3Gvl6Q1fZ7g,Swedish House Mafia,Two More Lonely People,spotify:track:0PbkWS1I4lJVVU6Atx2fuN,Miley Cyrus
7,Jumper - Radio Edit,spotify:track:5reOHWRm65rE1MEk6jTMAG,Hardwell,Gave Up - Open My Eyes,spotify:track:2uibtoCm0gT65lfzmdr7M6,Coil
8,Sweet Dreams - Timo Juuti & Hector 87 Remix,spotify:track:5sIeuXoQAfy8Ll3FdEkXRi,Avicii,"I'm A Wonderful Thing, Baby",spotify:track:4pTHRUj33iMfczO9r2Vzjt,Kid Creole And The Coconuts
9,Blue,spotify:track:6cZ6H2Jq4LSOqJs9pTZpi0,Calvin Harris,Our Deal,spotify:track:6tXV8dyGxxuRvKSILGo5rx,Best Coast


## Precision and Recall on 100 Random Playlists

#### Picks 100 random playlists

In [18]:
mylist = track_elems.pid.unique()

import random

sample_size = 100
sorted_sample = [mylist[i] for i in sorted(random.sample(range(len(mylist)), sample_size))]

df_dict = {name: track_elems.loc[track_elems['pid'] == name] for name in sorted_sample}

#### Sample of one of the playlists

In [19]:
df_dict[sorted_sample[99]].head()

,pid,pos,track_uri,COUNT(*)
58479444,742445,0,spotify:track:2teVg77oXxa7UOSochwdEo,37
60090966,742445,1,spotify:track:0sCPSj1zmc6utsy0czb54e,26
60614804,742445,8,spotify:track:5x2F5E0iZKGsUF3UPxMKTQ,50
61823365,742445,9,spotify:track:6xDH1AopZEPEAyVR1Ji9In,46
61888488,742445,2,spotify:track:1kcb0KVXfQBESU9p7d47nP,32


#### Loop that runs recommendation system for all 100 playlists

In [21]:
x = 0
while x < 100:
    Recommender_System(df_dict[sorted_sample[x]])
    x += 1

[[1063304 1140278 1237829  351273 2230239 1780340  744915  149795  687533
  2089364 1863179 1325160 1926064  646357  836667  407627  629311 1203287
  1006032 2055365  148630  621110  731813 1783704 1914462 1446818 2208702
  1511425 2208957  807870 2006208 1305110  120362 1034486   27860  755576
   731800  984971  744423 2060926  423483  326187  988272  116296  963924
  1255218  271932 2222316 1961963  795898  437492  583436  228896 1361709
  1924914 1388699 1404614]]

Precision: 0.0

Recall: 0.0
[[2166967 2096781 2050382  383806 1342862 1892157  548324 1757268 1678732
  1443476 2083578 1482754  131560  472433  453758  983985 1418191 1107524
  1502944 1123776  755327 1759419 1176387 1576090 1376008 1092936 1652546
  1155640  980924 2040693  166848 1864178 1349756 2208957  602332 1144888
  1946576 1545734 2000974 2180997 1665105 2182240  183512 1043949 1086133
  1724485 1149935 1522001 1133549  923832 1179748  552561  326354 1285823
    47718 1465544  756530  181031  514248 2088745  3990


Precision: 0.0

Recall: 0.0
[[ 379013 1088378  352834 1792054 1630943 2163667  138902  919279 1025121
  1491805 1756650  791672 1089377 1680252 1774969 1751163   29937 1198579
  1563491 1902197 1318600  225822 2031577  996359 1781430  862108 2074356
  2214769 2145813 1302555 1274705 1442697 1311089 1086141  166035  667456
   636802 1353432 1252175 1034031  992081 1550024 1335845 1730206   95782
   524230  560272 2092874 1092941 1555138]]

Precision: 0.0

Recall: 0.0
[[1342168 1883515 1553379 2059030  435433 1666491 2144393    1146 1802579
  1840969 1976313   69711 2034432  864175 2024654 1689593 1344127  166774
  1306328 1742556  425299 1309210  645729 1439684 1524367  791589 2145119
    58911 1826369  640389 1734885 2139649 1264154  560442 2235880 1889880
  2091027  270145 2195043  128679 1632972 1339886 1355782  753328 1766292
   986246  669327 1804986  253487  884888 1386001  421049  669235  641438
   981628 2021387 1697436  112035 1695270  951927 2053244 1377726  411025
   634756 


Precision: 0.0385

Recall: 0.0385
[[ 441800  441800 1309264   73731   73731 1565187 1786836  542857  690121
   277075 1167740  844152  592945 1794456 2171618 1352434 1842997 1904149
  1628618  652635  760313 1615454 2120386 1100985  305230 1027620  179959
   179959 1638504 1453690  575537 1903680 1903680 1266622 1564053 1105657
  1834485 1676894 1334149 1061464   52084 1160509 2193038 1343201 1228885
  2063622  741828  741828  231555 1535003  946708  351858   47571  883564
   999762 1452158 1255363 1820965 2042447 1296717 1560763 1939684 1939684
  1824613  916433  744858  375570 1986458 1150391 1898727    1792 1562374
  1562374 1281310 1900536  922222  284361 1895900 2247367 2247367 1946514
  1677301  469008 2095018 2060785  645729 2205957  892018 2062626]]

Precision: 0.0

Recall: 0.0
[[ 945703  982144  738825 1864201 1163236  362039  554257 1721157  185682
    80446 2219038  674907 1176387  616525 2228034  865476 1819821 1818403
   867729  319722 1082820 1693750 1454913  163695 1941


Precision: 0.0

Recall: 0.0
[[ 775189 1342451  254340 1197087  714954 1645774 1296331  632435  710535
   537344 1819684 1499988  116507 1546593 1752011 1843408  874592  168300
  2229790 2047291 2155898  516167 1887624 1463281 2069251  332559  460118
   624023 1021735  682468  654857 1798660 1013376  331050 1434147 1893066
  1545349  715599 1221049  601077 1471609 1179943 1350163  540675 2217082
  1993007 1929080 1927595  631056  498104 1015196  775207  530577  956180
  1312792 1229672 1284691 2116042 1851356  516250 1782184  870508  406699
  1786092 1391207 1741195  564147 1962971  491773]]

Precision: 0.0

Recall: 0.0
[[1916088 1600952   31166  634259  260884 1578251 1906881 1872265 1033015
   819608  793570  324775  246594  992761  382818 1511644  752396]]

Precision: 0.0

Recall: 0.0
[[ 576334 1233873 1233873 1336463 1102731  793794  984989  799740 2177921
  1584413 2118362 1547979 1083477 1392873 1851703 1317021 2126309 2163906
  1271083   89696  543761 1257036  848671 2231184  10


Precision: 0.0

Recall: 0.0
[[ 601596 1044537 1805972  484103 1272948  469582 2096237  387635  763051
   148307  482206  354897 1255875  898714 1739180 1487860  162879  692881
   705300]]

Precision: 0.0

Recall: 0.0
[[ 893387 1843977   83689  516830 1853527 1328299 1645199 2088607  203613
  1357331  402207 1058725 1049071 2108722 1397039 1544731 2028326 1531612
    56723   83773 1993507 1194899  786859  988471 1401726 1020998 1103346
  1625241 1218757 1281552  619428 1702378 1979045 1821431 1062142 1734974
   730838 1910086 1410488  400388 2108898 2184514  886855 1480856]]

Precision: 0.0

Recall: 0.0
[[1802483  479369 1222121  776169  984302  762938  435007 1552096 1708941
  1000699 1712896  504684  681695 1103162 1506499 2222007  447560 1938045
  1678744]]

Precision: 0.0

Recall: 0.0
[[2248352 2250722 2015331 1161096  211905 1176387 1059564  269840  597397
   119424  653945  610520 1818403  666771 1521799  300409 2254455 1508273
   243863 1255380  555882  100879]]

Precision: 0.0



Precision: 0.0

Recall: 0.0
[[1414381 1236797  863939 1374826  655636 2029448 1334933  232920 1846046
  1549512]]

Precision: 0.0

Recall: 0.0


In [22]:
print(Precision)
np_Precision = np.array(Precision)
print('\nMean Precision = '+str(np_Precision.mean())+'\n')

print(Recall)
np_Recall = np.array(Recall)
print('\nMean Recall = '+str(np_Recall.mean()))

[0.0, 0.0, 0.0286, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0167, 0.02, 0.0, 0.0, 0.0526, 0.0, 0.0167, 0.0, 0.0455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0145, 0.0, 0.0, 0.0, 0.0606, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1667, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0167, 0.0, 0.0, 0.0444, 0.0, 0.0339, 0.0, 0.0385, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0196, 0.0, 0.0, 0.0, 0.0278, 0.0, 0.0, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Precision = 0.003848

# Conclusion

We can see that there is a big improvement when adding popularity to the recommender system

<br>- With recommender System getting a song correct in 12 of the 100 playlists

<br>- Although these predictions do not have very strong precision and recall values this does not mean they are bad recommendations

<br>- As the aim of this project is a 'Playlist Continuation System' rather than a 'Playlist Predictor' these results are very good. 

<br>- After examining the <a href='#pop_djs'>Popular DJ</a> recommendations we can see some good results.
    - The Recommender System gave back another song by one of the artists Calvin Harris 
    - It also returned other songs from popular DJs

<br>- After adding a PCA reduced cluster values these precision,recall and general recommendations will improve.